In [1]:
import os
from glob import glob
import random
import time
from tqdm import tqdm
from PIL import Image

import tensorflow as tf
import tensorflow_hub as hub

import pandas as pd

def load_img(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    return img

def analyse_picture(file, detector):
    img = load_img(file)
    converted_img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]
    result = detector(converted_img)
    result = {key:value.numpy() for key,value in result.items()}
    dict_result = {
        'classes' : [elt.decode("utf-8")  for elt in result['detection_class_entities'].tolist()],
        'detection_boxes' : result['detection_boxes'].tolist(),
        'detection_scores' : result['detection_scores'].tolist()
    }
    dfp_result = pd.DataFrame(dict_result)
    return dfp_result

def crop_image(file, detector, debug=False):
    new_file = None
    dfp_results = analyse_picture(file, detector)
    dfp_objects = dfp_results[(dfp_results['detection_scores'] >= 0.95) & (dfp_results['classes'].isin(['Car']))].reset_index(drop=True)
    if debug:
        print(dfp_objects)
    if len(dfp_objects) == 1:
        infos_car = dfp_objects.iloc[0].to_dict()
        if debug:
            print('Car cropped')
            print(infos_car)
        ymin, xmin, ymax, xmax = infos_car['detection_boxes']
        img = Image.open(file)
        width, height = img.size
        crop_box = (xmin * width, ymin * height, xmax * width, ymax * height)
        crop_box = map(int, crop_box)
        img_cropped = img.crop((crop_box))

        new_file = file.replace("_raw", "_cropped")
        if debug:
            print(new_file)
        os.makedirs(os.path.dirname(new_file), exist_ok=True)
        img_cropped.save(new_file)
    else:
        if debug:
            print('Nothing to crop')
            
    return new_file

# Print Tensorflow version
print(tf.__version__)

# Check available GPU devices.
print("The following GPU devices are available: %s" % tf.test.gpu_device_name())

2.6.0
The following GPU devices are available: /device:GPU:0


2021-09-15 21:49:38.997048: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-09-15 21:49:39.040102: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-15 21:49:39.046906: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-15 21:49:39.047156: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [2]:
# Load the model
module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"
detector = hub.load(module_handle).signatures['default']

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
2021-09-13 11:01:04.397348: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


In [12]:
parent_folder = "./data/pictures/testdata/"
child_folder = "20210913_100_raw"
PATH = parent_folder + child_folder
files = [y for x in os.walk(PATH) for y in glob(os.path.join(x[0], '*.jpg')) if '_checkpoints' not in y]
print(len(files))

728


In [7]:
file = random.choice(files)
crop_image(file, detector)

'./data/pictures/testdata/20210913_100_cropped/cadillac/4_38.jpg'

In [13]:
for file in tqdm(files):
    new_file = crop_image(file, detector)

100%|███████████████████████████████████████| 728/728 [1:07:51<00:00,  5.59s/it]
